In [ ]:
%load_ext autoreload
%autoreload 2

In [40]:
import torch
from pyrad.fields.occupancy_fields.occupancy_grid import DensityGrid
from pyrad.cuda.ray_sampler import NGPSpacedSampler

#  lego [-1.5, 1.5]

density_field = DensityGrid(num_cascades=2, resolution=32)
sampler = NGPSpacedSampler(num_samples=128, density_field=density_field)


In [32]:
import torch
from pyrad.fields.occupancy_fields.occupancy_grid import DensityGrid


device = "cuda:3"

# there is a sphere at (0.5, 0.5, 0.5) with radius 0.5
def density_eval_func(x):
    density = (x.float() - 0.5).norm(dim=-1) < 0.5
    return density[..., None].float()
density_grid = DensityGrid(4, 4).to(device)
density_grid.reset()

In [38]:
# update_density_grid()
tmp_grid = -torch.ones_like(density_grid.density_grid)
cells = density_grid.get_all_cells()
for mip_level in range(density_grid.num_cascades):
    mip_scale = 2 ** (-mip_level)
    indices, coords = cells[mip_level]
    x = (coords + torch.ones_like(coords.float()) / 2.) / density_grid.resolution
    x = (x - 0.5) / mip_scale + 0.5
    density = density_eval_func(x).squeeze(-1)
    tmp_grid[mip_level, indices] = density
    print (mip_scale)
    print (x.unique())

    # break

1
tensor([0.1250, 0.3750, 0.6250, 0.8750], device='cuda:3')
0.5
tensor([-0.2500,  0.2500,  0.7500,  1.2500], device='cuda:3')
0.25
tensor([-1.,  0.,  1.,  2.], device='cuda:3')
0.125
tensor([-2.5000, -0.5000,  1.5000,  3.5000], device='cuda:3')


In [ ]:
import plotly.graph_objects as go



In [ ]:
grid_x, grid_y, grid_z = density_grid.grid_coords.split([1, 1, 1], dim=-1)
values = density_grid.grid_coords.norm(dim=-1) > 

In [ ]:
import plotly.graph_objects as go
import numpy as np
X, Y, Z = np.mgrid[-1:1:30j, -1:1:30j, -1:1:30j]
values =    np.sin(np.pi*X) * np.cos(np.pi*Z) * np.sin(np.pi*Y)

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=-0.1,
    isomax=0.8,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=21, # needs to be a large number for good volume rendering
    ))
fig.show()